In [2]:
import numpy as np
import pytest

import sys
import os
sys.path.insert(0, os.path.abspath('../src'))
from wale.CosmologyModel import *



# pick some fiducial cosmology
COSMO = Cosmology_function(h=0.7, Oc=0.25, Ob=0.05, sigma8=0.8)

# 1) Test E(z) and H(z)
def test_E_at_zero_is_one():
    assert np.isclose(COSMO._E(0.0), 1.0)

def test_H_at_zero_equals_H0():
    assert np.isclose(COSMO.get_H(0.0), COSMO.H0)

# 2) χ(z) monotonicity and zero point
def test_chi_zero_is_zero():
    assert np.isclose(COSMO.get_chi(0.0), 0.0)

def test_chi_increases_with_z():
    zs = np.linspace(0, 2, 5)
    chis = COSMO.get_chi(zs)
    # check strictly increasing
    assert np.all(np.diff(chis) > 0)

# 3) Inversion consistency: z(χ(z)) ≃ z
@pytest.mark.parametrize("z", [0.1, 0.5, 1.0, 2.0])
def test_inversion_roundtrip(z):
    chi = COSMO.get_chi(z)
    z_recov = COSMO.get_z_from_chi(chi)
    assert np.isclose(z, z_recov, atol=1e-5)

# 4) Nonlinear P(k): shape and positivity
def test_nonlinear_pk_shape_and_positive():
    z = 0.5
    # sample k array
    ks = np.logspace(-3, 1, 50)
    P = COSMO.get_nonlinear_pk(z, ks=ks)
    assert P.shape == ks.shape
    assert np.all(P > 0)

# 5) Lensing weight array—basic sanity checks
def test_lensing_weight_array_zero_beyond_source():
    # choose a source at z=1, compute chi_source
    chi_s = COSMO.get_chi(1.0)
    # chis beyond source
    chis = np.array([chi_s * 1.1, chi_s * 1.5])
    z_vals, W = COSMO.get_lensing_weight_array(chis, chi_s)
    # beyond the source lensing weight should be zero
    assert np.allclose(W, 0.0)

def test_lensing_weight_normalization_behaviour():
    # for chis very small, lensing weight ∼0
    chi_s = COSMO.get_chi(1.0)
    chis = np.array([0.0, chi_s * 0.01])
    z_vals, W = COSMO.get_lensing_weight_array(chis, chi_s)
    assert np.isclose(W[0], 0.0)
    # at small but nonzero chi, W should be > 0
    assert W[1] > 0

# 6) nz‐based lensing weight vs simple array version
def test_nz_based_vs_array_version_consistency():
    # build a toy n(z): uniform from z=0 to z=1
    z_nz = np.linspace(0, 1, 1001)
    n_z = np.ones_like(z_nz)
    # choose chis array
    chis = np.linspace(0, COSMO.get_chi(1.0), 5)
    # compute both
    z_arr1, W1 = COSMO.get_lensing_weight_array(chis, COSMO.get_chi(1.0))
    z_arr2, W2 = COSMO.get_lensing_weight_array_nz(chis, z_nz, n_z)
    # they won’t be identical, but shapes must match and W2≥0
    assert W2.shape == W1.shape
    assert np.all(W2 >= 0)

# 7) Edge‐cases & errors
def test_inversion_out_of_bounds_raises():
    # chi at z_max * 1.1 triggers ValueError
    chi_too_big = COSMO.get_chi(10.0) * 1.1
    with pytest.raises(ValueError):
        _ = COSMO.get_z_from_chi(chi_too_big, z_max=5.0)

def test_missing_normalization_parameters():
    # neither sigma8 nor As → error
    with pytest.raises(ValueError):
        Cosmology_function(h=0.7, Oc=0.25, Ob=0.05)



In [4]:
pytest -q


NameError: name 'q' is not defined